In [1]:
%pip install PyPDF2 spacy nltk

Note: you may need to restart the kernel to use updated packages.


# Análise de Artigos Científicos

In [2]:
import os
import PyPDF2
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter, defaultdict
import spacy
import subprocess
import sys

try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Modelo 'en_core_web_sm' não encontrado. Baixando agora...")
    subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load("en_core_web_sm")

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

PASTA_PDFS = "artigos"

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gustavognz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Funções Comuns

In [4]:
def extrair_texto_pdf_limpo(path):
    doc = PyPDF2.PdfReader(open(path, "rb"))
    total_paginas = len(doc.pages)

    linhas_por_pagina = []
    for pagina in doc.pages:
        texto = pagina.extract_text() or ""
        linhas = [l.strip() for l in texto.splitlines() if l.strip()]
        linhas_por_pagina.append(linhas)

    contador_linhas = defaultdict(int)
    for linhas in linhas_por_pagina:
        linhas_unicas = set(linhas)
        for linha in linhas_unicas:
            contador_linhas[linha] += 1

    limiar = 0.8 * total_paginas  # descarta linhas que aparecem em 80%+ das páginas

    texto_limpo = ""
    for linhas in linhas_por_pagina:
        linhas_filtradas = [l for l in linhas if contador_linhas[l] < limiar]
        texto_limpo += " ".join(linhas_filtradas) + " "

    return texto_limpo

In [5]:
def preprocessar_texto(texto):
    doc = nlp(texto.lower())
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.text not in stop_words]
    return tokens

In [ ]:
def remover_referencias(texto):
    partes = re.split(r"\breferences\b", texto, flags=re.IGNORECASE)
    return partes[0], partes[1] if len(partes) > 1 else ""

### Etapa 1: Leitura, pré-processamento e top 10 unigramas

In [21]:
todos_unigramas = []
referencias = []

for arquivo in os.listdir(PASTA_PDFS):
    if not arquivo.endswith(".pdf"):
        continue
    caminho = os.path.join(PASTA_PDFS, arquivo)
    print(f"Lendo {arquivo}...")
    texto = extrair_texto_pdf_limpo(caminho)

    corpo, ref = remover_referencias(texto)
    referencias.append(ref)

    tokens = preprocessar_texto(corpo)
    todos_unigramas.extend(tokens)

cont_uni = Counter(todos_unigramas)

print("\nTop 10 Unigramas:")
for palavra, freq in cont_uni.most_common(10):
    print(f"{palavra}: {freq}")

print("\n\nReferências encontradas:\n")

for i, ref in enumerate(referencias, 1):
    print(f"Referência {i}:\n\n")
    print(ref.strip() + "\n\n")
    print("-" * 40 + "\n\n")

Lendo A simplified approach for dynamic security risk management in connected and autonomous vehicles.pdf...
Lendo Towards an integrated privacy protection framework for IoT Contextualising regulatory requirements with industry best practices.pdf...
Lendo Emerging privacy challenges and approaches in CAV systems.pdf...
Lendo Automation_in_Agriculture_and_IoT.pdf...
Lendo Cloud for IoT - A survey of technologies and security features of public cloud IoT solutions.pdf...
Lendo Genoma_Distributed_Provenance_as_a_Service_for_IoT-based_Systems.pdf...
Lendo Secure - zero touch device onboarding.pdf...
Lendo Towards an understanding of emerging cyber security threats in mapping the IoT.pdf...
Lendo IoT_Challenges_and_Issues_in_Indian_Perspective.pdf...

Top 10 Unigramas:
iot: 319
datum: 244
privacy: 242
device: 196
system: 172
use: 142
security: 134
risk: 126
attack: 110
service: 110


Referências encontradas:

Referência 1:


[1] T. Maughan, "No One's Driving: Autonomous Vehicles Will Reshap

### Etapa 2: Extrair objetivo, problema, método e contribuição

In [ ]:
padroes = {
    "objetivo": re.compile(r"\b(we|objective|aim|purpose|goal)(s)? (propose(s)?|present(s)?|of|this|the)? (study|article|research|paper)?\b", re.IGNORECASE),
    "problema": re.compile(r"\b(problem|issue|challenge)(s)?\b", re.IGNORECASE),
    "metodo": re.compile(r"\b(method|methodology|interview(s)?|survey(s)?|content analysis|case study|experiment|approach|technique(s)?|) (were|was)? (conducted|used|realized|taken)?\b", re.IGNORECASE),
    "contribuicao": re.compile(r"\b(we|our|this|the)? (study|article|research|paper)? (contribute(s)?|contribution(s)?|add(s)?|) (to)?\b", re.IGNORECASE),
}

resultados = []

for arquivo in os.listdir(PASTA_PDFS):
    if not arquivo.endswith(".pdf"):
        continue
    caminho = os.path.join(PASTA_PDFS, arquivo)
    print(f"Extraindo infos de {arquivo}...")

    texto = extrair_texto_pdf_limpo(caminho)
    corpo, _ = remover_referencias(texto)

    doc = nlp(corpo) # Processamento com spaCy

    objetivo = ""
    problema = ""
    metodo = ""
    contribuicao = ""

    # Verifica padrões em cada sentença
    for sentenca in doc.sents:
        if padroes["objetivo"].search(sentenca.text):
            objetivo += sentenca.text.strip() + ";;\n"
        if padroes["problema"].search(sentenca.text):
            problema += sentenca.text.strip() + ";;\n"
        if padroes["metodo"].search(sentenca.text):
            metodo += sentenca.text.strip() + ";;\n"
        if padroes["contribuicao"].search(sentenca.text):
            contribuicao += sentenca.text.strip() + ";;\n"


    resultados.append({
        "arquivo": arquivo,
        "objetivo": objetivo,
        "problema": problema,
        "metodo": metodo,
        "contribuicao": contribuicao,
    })

print("\n\nResultados extraídos:\n")
for info in resultados:
    print(f"Arquivo: {info['arquivo']}\n")
    print("Objetivo:\n")
    print(f"{info['objetivo']}\n\n")
    print("Problema:\n")
    print(f"{info['problema']}\n\n")
    print("Método:\n")
    print(f"{info['metodo']}\n\n")
    print("Contribuição:\n")
    print(f"{info['contribuicao']}\n\n")
    print("-" * 40 + "\n\n")


Extraindo infos de A simplified approach for dynamic security risk management in connected and autonomous vehicles.pdf...
Extraindo infos de Cloud for IoT - A survey of technologies and security features of public cloud IoT solutions.pdf...
Extraindo infos de Emerging privacy challenges and approaches in CAV systems.pdf...
Extraindo infos de Secure - zero touch device onboarding.pdf...
Extraindo infos de Towards an integrated privacy protection framework for IoT Contextualising regulatory requirements with industry best practices.pdf...
Extraindo infos de Towards an understanding of emerging cyber security threats in mapping the IoT.pdf...


Resultados extraídos:

Arquivo: A simplified approach for dynamic security risk management in connected and autonomous vehicles.pdf

Objetivo:

We propose a simplified approach to identify the most critical threats faced by CAVs’ through monitoring the security context of both the CAVs and the environments they operate in.;;
x We propose a method t

### Etapa 3: Salvar dados extraídos

In [ ]:
def etapa3_salvar_dados(dados_extraidos):
    with open("dados_extraidos.txt", "w", encoding="utf-8") as f:
        for info in dados_extraidos:
            f.write(f"Arquivo: {info['arquivo']}\n")
            f.write("Objetivo:\n")
            f.write(f"{info['objetivo']}\n\n")
            f.write("Problema:\n")
            f.write(f"{info['problema']}\n\n")
            f.write("Método:\n")
            f.write(f"{info['metodo']}\n\n")
            f.write("Contribuição:\n")
            f.write(f"{info['contribuicao']}\n\n")
            f.write("-" * 40 + "\n\n")



### Execução Principal

In [ ]:
# Etapa 1
contagem_unigramas, referencias = etapa1_identificar_termos_mais_citados()
salvar_referencias(referencias)

# # Etapa 2
infos = etapa2_extrair_info_artigos()

# Etapa 3
etapa3_salvar_dados(infos)

# print("\nProcessamento concluído. Dados extraídos salvos em dados_extraidos.txt")